In [1]:
# SWAMI KARUPPASWAMI THUNNAI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [7]:
dataset = pd.read_csv("hate.csv", encoding="utf8")

In [8]:
dataset.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech:confidence,_created_at,orig__golden,orig__last_judgment_at,orig__trusted_judgments,orig__unit_id,orig__unit_state,_updated_at,orig_does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech_gold,does_this_tweet_contain_hate_speech_gold_reason,does_this_tweet_contain_hate_speechconfidence,tweet_id,tweet_text
0,853718217,True,golden,86,NaN,The tweet uses offensive language but not hate...,0.6013,NaN,True,NaN,0.0,615561535.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,1.666196e+09,Warning: penny boards will make you a faggot
1,853718218,True,golden,92,NaN,The tweet contains hate speech,0.7227,NaN,True,NaN,0.0,615561723.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,4.295121e+08,Fuck dykes
2,853718219,True,golden,86,NaN,The tweet contains hate speech,0.5229,NaN,True,NaN,0.0,615562039.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,3.956238e+08,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...
3,853718220,True,golden,98,NaN,The tweet contains hate speech,0.5184,NaN,True,NaN,0.0,615562068.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,4.975147e+08,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill..."
4,853718221,True,golden,88,NaN,The tweet uses offensive language but not hate...,0.5185,NaN,True,NaN,0.0,615562488.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,5.889236e+08,@Zhugstubble You heard me bitch but any way I'...


In [9]:
X = dataset["tweet_text"]

In [30]:
y = dataset["does_this_tweet_contain_hate_speech"]

In [14]:
from nltk.tokenize import TweetTokenizer
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


def preprocess(tweet_text):
    """
    Will preprocess the tweets like remove the usernames, hashtags, urls and
    will preprocess the content suitable for NLP.
    :return: preprocessed tweet
    """
    # convert the tweet to the lower case
    tweet_text = tweet_text.lower()
    tokenizer = TweetTokenizer()
    words = tokenizer.tokenize(tweet_text)
    preprocessed_words = []
    for word in words:
        if word.startswith("@") or word.startswith("#") or word.startswith("https://") or word.startswith("http://"):
            pass
        else:
            preprocessed_words.append(word)
    # remove the stop words
    stopwords_removed = [word for word in preprocessed_words if not word in stopwords.words("english")]
    # make sure they are words
    punc_remover = lambda word: re.sub("[^A-Za-z]", " ", word)
    # get the pure words without punctuation
    pure_words = list(map(punc_remover, stopwords_removed))
    # Stem the words
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in pure_words]
    # strip the words
    stripper = lambda word: word.strip()
    stemmed_words = list(map(stripper, stemmed_words))
    stemmed_words = filter(None, stemmed_words)
    # Get our processed tweet
    processed_tweet = " ".join(stemmed_words)
    return processed_tweet

In [17]:
corpus = []
for i in X:
    result = preprocess(i)
    corpus.append(result)
    
    

In [18]:
corpus[0]

'warn penni board make faggot'

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()

In [21]:
print("Completed")

Completed


In [22]:
X[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [31]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [32]:
from sklearn.linear_model import LogisticRegression

In [34]:
classifier = LogisticRegression()
classifier.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
from sklearn.externals import joblib

In [36]:
joblib.dump(cv, "hate_cv.pkl")

['hate_cv.pkl']

In [38]:
joblib.dump(classifier, "hate.pkl")

['hate.pkl']